When point cloud of both are available and there are no missing point clouds, we can use principal components to approximately align two point clouds

# Part 1: Reading point cloud in dictionary, applying known arbitrary rotation and translation to the point cloud and saving it in another dictionary

In [1]:
file_name = "Part1_STL_10_pow_4_ASCII.txt"

In [2]:
with open(file_name) as file_object:
    lines = file_object.readlines()

In [3]:
data = {}

In [4]:
i = 1
for line in lines:
    items = line.split(",")
    x, y, z = float(items[0]), float(items[1]), float(items[2])
    list_x_y_z = [x, y, z]
    data[i] = list_x_y_z
    i+=1

In [5]:
data[1]

[0.97175932, 0.27299145, 0.60483378]

In [6]:
len(data)

10029

In [7]:
data[10029]

[0.91336119, 0.46265504, 0.37359932]

Applying translation

In [8]:
transformed_data = {}

In [9]:
tx, ty, tz = 1.0, 2.0, 3.0

In [10]:
for key, value in data.items():
    transformed_data[key] = [value[0]+tx, value[1]+ty, value[2]+tz]    

In [11]:
transformed_data[1]

[1.9717593199999999, 2.27299145, 3.60483378]

In [12]:
transformed_data[10029]

[1.91336119, 2.46265504, 3.37359932]

Applying rotation of 45 degress along z-axis of coordinate frame, just to keep it simple.

In [13]:
rotation_z = [[0.717, -0.717, 0],
             [0.717, 0.717, 0],
             [0, 0, 1]]

In [16]:
for key, value in transformed_data.items():
    transformed_data[key] = [rotation_z[0][0]*value[0] + rotation_z[0][1]*value[1] + rotation_z[0][2]*value[2],
                            rotation_z[1][0]*value[0] + rotation_z[1][1]*value[1] + rotation_z[1][2]*value[2],
                            rotation_z[2][0]*value[0] + rotation_z[2][1]*value[1] + rotation_z[2][2]*value[2]]

In [17]:
transformed_data[1]

[-0.21598343721000024, 3.04348630209, 3.60483378]

In [18]:
transformed_data[10029]

[-0.39384369045, 3.1376036369099998, 3.37359932]

Original and transformed data visualization

In [19]:
import numpy as np
import ipyvolume as ipv

In [20]:
x, y, z = np.empty(0), np.empty(0), np.empty(0)

In [21]:
for key, value in data.items():
        x = np.append(x, value[0])
        y = np.append(y, value[1])
        z = np.append(z, value[2])

In [23]:
x1, y1, z1 = np.empty(0), np.empty(0), np.empty(0)

In [24]:
for key, value in transformed_data.items():
        x1 = np.append(x1, value[0])
        y1 = np.append(y1, value[1])
        z1 = np.append(z1, value[2])

In [25]:
fig = ipv.figure()
scatter = ipv.scatter(x, y, z, color = 'red', size=0.75, marker='sphere')
scatter1 = ipv.scatter(x1, y1, z1, color = 'blue', size=0.75, marker='sphere')
ipv.show()

The red is reference point cloud and blue is transformed point cloud.

# Part 2: Calculate mean of both point clouds and align the mean point, this should equal the translation part of translation and rotation combined

In [26]:
mean_x, mean_y, mean_z = 0.0, 0.0, 0.0
mean_x1, mean_y1, mean_z1 = 0.0, 0.0, 0.0

In [27]:
for key, value in data.items():
    mean_x += value[0]
    mean_y += value[1]
    mean_z += value[2]
    
mean_x = mean_x / len(data)
mean_y = mean_y / len(data)
mean_z = mean_z / len(data)

In [28]:
mean_x

0.677856793295442

In [29]:
for key, value in transformed_data.items():
    mean_x1 += value[0]
    mean_y1 += value[1]
    mean_z1 += value[2]
    
mean_x1 = mean_x1 / len(data)
mean_y1 = mean_y1 / len(data)
mean_z1 = mean_z1 / len(data)

In [30]:
mean_x1

-0.383403964464602

In [32]:
translation = [mean_x - mean_x1, mean_y - mean_y1, mean_z - mean_z1]
translation

[1.061260757760044, -2.576860250042692, -3.000000000000017]

In [33]:
for key, value in transformed_data.items():
    value[0] = value[0] + translation[0]
    value[1] = value[1] + translation[1]
    value[2] = value[2] + translation[2]

data visualization

In [34]:
x, y, z = np.empty(0), np.empty(0), np.empty(0)
for key, value in data.items():
        x = np.append(x, value[0])
        y = np.append(y, value[1])
        z = np.append(z, value[2])

In [35]:
x1, y1, z1 = np.empty(0), np.empty(0), np.empty(0)
for key, value in transformed_data.items():
        x1 = np.append(x1, value[0])
        y1 = np.append(y1, value[1])
        z1 = np.append(z1, value[2])

In [36]:
fig = ipv.figure()
scatter = ipv.scatter(x, y, z, color = 'red', size=0.75, marker='sphere')
scatter1 = ipv.scatter(x1, y1, z1, color = 'blue', size=0.75, marker='sphere')
ipv.show()

The mean method seems to work.

# Part 3: Compute eigen values and eigen vectors of both the point clouds using covariance matrix of point clouds

In [38]:
covariance_matrix = np.zeros((3,3))
cov_x_x, cov_y_y, cov_z_z, cov_x_y, cov_x_z, cov_y_z = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
len_data = len(data)
for key, value in data.items():
    cov_x_x += (value[0] - mean_x)**2 / (len_data - 1)
    cov_y_y += (value[1] - mean_y)**2 / (len_data - 1)
    cov_z_z += (value[2] - mean_z)**2 / (len_data - 1)
    cov_x_y += (value[0] - mean_x)*(value[1] - mean_y) / (len_data - 1)
    cov_x_z += (value[0] - mean_x)*(value[2] - mean_z) / (len_data - 1)
    cov_y_z += (value[1] - mean_y)*(value[2] - mean_z) / (len_data - 1)
            
covariance_matrix[0,0] = cov_x_x
covariance_matrix[1,1] = cov_y_y
covariance_matrix[2,2] = cov_z_z
covariance_matrix[0,1] = cov_x_y
covariance_matrix[1,0] = cov_x_y
covariance_matrix[0,2] = cov_x_z
covariance_matrix[2,0] = cov_x_z
covariance_matrix[1,2] = cov_y_z
covariance_matrix[2,1] = cov_y_z

In [39]:
eigenValue, eigenVector = np.linalg.eig(covariance_matrix)

In [41]:
eigenValue, eigenVector

(array([0.08824332, 0.00929945, 0.001606  ]),
 array([[-0.81375597, -0.41443035, -0.40749074],
        [-0.56384187,  0.39281401,  0.72648434],
        [ 0.14100909, -0.82094131,  0.55332793]]))

In [42]:
covariance_matrix1 = np.zeros((3,3))
cov_x_x1, cov_y_y1, cov_z_z1, cov_x_y1, cov_x_z1, cov_y_z1 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
len_data = len(transformed_data)
for key, value in transformed_data.items():
    cov_x_x1 += (value[0] - mean_x)**2 / (len_data - 1)
    cov_y_y1 += (value[1] - mean_y)**2 / (len_data - 1)
    cov_z_z1 += (value[2] - mean_z)**2 / (len_data - 1)
    cov_x_y1 += (value[0] - mean_x)*(value[1] - mean_y) / (len_data - 1)
    cov_x_z1 += (value[0] - mean_x)*(value[2] - mean_z) / (len_data - 1)
    cov_y_z1 += (value[1] - mean_y)*(value[2] - mean_z) / (len_data - 1)
            
covariance_matrix1[0,0] = cov_x_x1
covariance_matrix1[1,1] = cov_y_y1
covariance_matrix1[2,2] = cov_z_z1
covariance_matrix1[0,1] = cov_x_y1
covariance_matrix1[1,0] = cov_x_y1
covariance_matrix1[0,2] = cov_x_z1
covariance_matrix1[2,0] = cov_x_z1
covariance_matrix1[1,2] = cov_y_z1
covariance_matrix1[2,1] = cov_y_z1

In [43]:
eigenValue1, eigenVector1 = np.linalg.eig(covariance_matrix1)
eigenValue1, eigenVector1

(array([0.09068051, 0.00163708, 0.0093851 ]),
 array([[-0.17694516, -0.79687337,  0.57765323],
        [-0.97438762,  0.22458847,  0.01134833],
        [ 0.13877743,  0.56085012,  0.81620339]]))

# Part 4: Align the point clouds using eigen vectors